In [1]:
# Load packages
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

# Load applications data
path = 'dataset/'
train = pd.read_csv(path + 'application_train.csv')
test = pd.read_csv(path + 'application_test.csv')

train.sample(frac=0.1, replace=True, random_state=1)
test.sample(frac=0.1, replace=True, random_state=1)

train.to_csv('train_sample.csv', index=False)
test.to_csv('test_sample.csv', index=False)

In [2]:
# Load packages
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.preprocessing import LabelEncoder, OneHotEncoder, RobustScaler
from sklearn.impute import SimpleImputer
from sklearn.pipeline import Pipeline
from sklearn.metrics import auc, roc_curve, roc_auc_score, make_scorer, fbeta_score
from sklearn.model_selection import cross_val_score, cross_val_predict, StratifiedKFold
from sklearn.calibration import calibration_curve, CalibratedClassifierCV
from xgboost import XGBClassifier
from xgboost import plot_importance
from hashlib import sha256
from imblearn.over_sampling import RandomOverSampler, SMOTE
from imblearn.under_sampling import RandomUnderSampler
from imblearn.pipeline import make_pipeline

# Plot settings
%matplotlib inline
%config InlineBackend.figure_format = 'svg'
sns.set()

# Load applications data
path = ''
train = pd.read_csv(path + 'train_sample.csv')
test = pd.read_csv(path + 'test_sample.csv')

# Merge test and train into all application data
train_o = train.copy()
train['Test'] = False
test['Test'] = True
test['TARGET'] = np.nan
app = train.append(test, ignore_index=True)

# Remove entries with gender = XNA
app = app[app['CODE_GENDER'] != 'XNA']

# Remove entries with income type = maternity leave
app = app[app['NAME_INCOME_TYPE'] != 'Maternity leave']

# Remove entries with unknown family status
app = app[app['NAME_FAMILY_STATUS'] != 'Unknown']

app['DAYS_EMPLOYED'].replace(365243, np.nan, inplace=True)

app['PROPORTION_LIFE_EMPLOYED'] = app['DAYS_EMPLOYED'] / app['DAYS_BIRTH']
app['INCOME_TO_CREDIT_RATIO'] = app['AMT_INCOME_TOTAL'] / app['AMT_CREDIT'] 
app['INCOME_TO_ANNUITY_RATIO'] = app['AMT_INCOME_TOTAL'] / app['AMT_ANNUITY']
app['INCOME_TO_ANNUITY_RATIO_BY_AGE'] = app['INCOME_TO_ANNUITY_RATIO'] * app['DAYS_BIRTH']
app['CREDIT_TO_ANNUITY_RATIO'] = app['AMT_CREDIT'] / app['AMT_ANNUITY']
app['CREDIT_TO_ANNUITY_RATIO_BY_AGE'] = app['CREDIT_TO_ANNUITY_RATIO'] * app['DAYS_BIRTH']
app['INCOME_TO_FAMILYSIZE_RATIO'] = app['AMT_INCOME_TOTAL'] / app['CNT_FAM_MEMBERS']

# Add indicator columns for empty values
for col in app:
    if col!='Test' and col!='TARGET':
        app_null = app[col].isnull()
        if app_null.sum()>0:
            app[col+'_ISNULL'] = app_null
            
# Label encoder
le = LabelEncoder()

# Label encode binary fearures in training set
for col in app: 
    if col!='Test' and col!='TARGET' and app[col].dtype==object and app[col].nunique()==2:
        if col+'_ISNULL' in app.columns: #missing values here?
            app.loc[app[col+'_ISNULL'], col] = 'NaN'
        app[col] = le.fit_transform(app[col])
        if col+'_ISNULL' in app.columns: #re-remove missing vals
            app.loc[app[col+'_ISNULL'], col] = np.nan
            
# Get categorical features to encode
cat_features = []
for col in app: 
    if col!='Test' and col!='TARGET' and app[col].dtype==object and app[col].nunique()>2:
        cat_features.append(col)

# One-hot encode categorical features in train set
app = pd.get_dummies(app, columns=cat_features)

# Hash columns
hashes = dict()
for col in app:
    hashes[col] = sha256(app[col].values).hexdigest()
    
# Get list of duplicate column lists
Ncol = app.shape[1] #number of columns
dup_list = []
dup_labels = -np.ones(Ncol)
for i1 in range(Ncol):
    if dup_labels[i1]<0: #if not already merged,
        col1 = app.columns[i1]
        t_dup = [] #list of duplicates matching col1
        for i2 in range(i1+1, Ncol):
            col2 = app.columns[i2]
            if ( dup_labels[i2]<0 #not already merged
                 and hashes[col1]==hashes[col2] #hashes match
                 and app[col1].equals(app[col2])): #cols are equal
                #then this is actually a duplicate
                t_dup.append(col2)
                dup_labels[i2] = i1
        if len(t_dup)>0: #duplicates of col1 were found!
            t_dup.append(col1)
            dup_list.append(t_dup)
        
# Merge duplicate columns
for iM in range(len(dup_list)):
    new_name = 'Merged'+str(iM)
    app[new_name] = app[dup_list[iM][0]].copy()
    app.drop(columns=dup_list[iM], inplace=True)
    print('Merged', dup_list[iM], 'into', new_name)
    
# Split data back into test + train
train = app.loc[~app['Test'], :]
test = app.loc[app['Test'], :]

train=train[['SK_ID_CURR','EXT_SOURCE_3','EXT_SOURCE_2','EXT_SOURCE_1','DAYS_EMPLOYED','AMT_INCOME_TOTAL','AMT_CREDIT','AMT_ANNUITY']]

train.to_csv('train_clean.csv', index=False)
test.to_csv('test_clean.csv', index=False)

/tmp/ipykernel_51528/2381566190.py:34: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  app = train.append(test, ignore_index=True)


Merged ['INCOME_TO_ANNUITY_RATIO_ISNULL', 'INCOME_TO_ANNUITY_RATIO_BY_AGE_ISNULL', 'CREDIT_TO_ANNUITY_RATIO_ISNULL', 'CREDIT_TO_ANNUITY_RATIO_BY_AGE_ISNULL', 'AMT_ANNUITY_ISNULL'] into Merged0
Merged ['PROPORTION_LIFE_EMPLOYED_ISNULL', 'DAYS_EMPLOYED_ISNULL'] into Merged1
Merged ['APARTMENTS_MODE_ISNULL', 'APARTMENTS_MEDI_ISNULL', 'APARTMENTS_AVG_ISNULL'] into Merged2
Merged ['BASEMENTAREA_MODE_ISNULL', 'BASEMENTAREA_MEDI_ISNULL', 'BASEMENTAREA_AVG_ISNULL'] into Merged3
Merged ['YEARS_BEGINEXPLUATATION_MODE_ISNULL', 'YEARS_BEGINEXPLUATATION_MEDI_ISNULL', 'YEARS_BEGINEXPLUATATION_AVG_ISNULL'] into Merged4
Merged ['YEARS_BUILD_MODE_ISNULL', 'YEARS_BUILD_MEDI_ISNULL', 'YEARS_BUILD_AVG_ISNULL'] into Merged5
Merged ['COMMONAREA_MODE_ISNULL', 'COMMONAREA_MEDI_ISNULL', 'COMMONAREA_AVG_ISNULL'] into Merged6
Merged ['ELEVATORS_MODE_ISNULL', 'ELEVATORS_MEDI_ISNULL', 'ELEVATORS_AVG_ISNULL'] into Merged7
Merged ['ENTRANCES_MODE_ISNULL', 'ENTRANCES_MEDI_ISNULL', 'ENTRANCES_AVG_ISNULL'] into Merged8

In [3]:
pd.set_option('display.max_columns', None)
train

,SK_ID_CURR,EXT_SOURCE_3,EXT_SOURCE_2,EXT_SOURCE_1,DAYS_EMPLOYED,AMT_INCOME_TOTAL,AMT_CREDIT,AMT_ANNUITY
0,100002,0.139376,0.262949,0.083037,-637.0,202500.0,406597.5,24700.5
1,100003,NaN,0.622246,0.311267,-1188.0,270000.0,1293502.5,35698.5
2,100004,0.729567,0.555912,NaN,-225.0,67500.0,135000.0,6750.0
3,100006,NaN,0.650442,NaN,-3039.0,135000.0,312682.5,29686.5
4,100007,NaN,0.322738,NaN,-3038.0,121500.0,513000.0,21865.5
...,...,...,...,...,...,...,...,...
307506,456251,NaN,0.681632,0.145570,-236.0,157500.0,254700.0,27558.0
307507,456252,NaN,0.115992,NaN,NaN,72000.0,269550.0,12001.5
307508,456253,0.218859,0.535722,0.744026,-7921.0,153000.0,677664.0,29979.0
307509,456254,0.661024,0.514163,NaN,-4786.0,171000.0,370107.0,20205.0


In [4]:
# Load packages
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import joblib
from sklearn.preprocessing import LabelEncoder, OneHotEncoder, RobustScaler
from sklearn.impute import SimpleImputer
from sklearn.pipeline import Pipeline
from sklearn.metrics import auc, roc_curve, roc_auc_score, make_scorer, fbeta_score
from sklearn.model_selection import cross_val_score, cross_val_predict, StratifiedKFold
from sklearn.calibration import calibration_curve, CalibratedClassifierCV
from xgboost import XGBClassifier
from xgboost import plot_importance
from hashlib import sha256
from imblearn.over_sampling import RandomOverSampler, SMOTE
from imblearn.under_sampling import RandomUnderSampler
from imblearn.pipeline import make_pipeline


pd.options.display.max_columns = None

# Plot settings
%matplotlib inline
%config InlineBackend.figure_format = 'svg'
sns.set()

# Load applications data
path = ''#'dataset/'
train = pd.read_csv(path + 'train_sample.csv')
test = pd.read_csv(path + 'test_sample.csv')
train_ids=train['SK_ID_CURR']
test_ids=test['SK_ID_CURR']


# Merge test and train into all application data
train_o = train.copy()
train['Test'] = False
test['Test'] = True
test['TARGET'] = np.nan
app = train.append(test, ignore_index=True)

# Remove entries with gender = XNA
app = app[app['CODE_GENDER'] != 'XNA']
# Remove entries with income type = maternity leave
app = app[app['NAME_INCOME_TYPE'] != 'Maternity leave']
# Remove entries with unknown family status
app = app[app['NAME_FAMILY_STATUS'] != 'Unknown']
app['DAYS_EMPLOYED'].replace(365243, np.nan, inplace=True)
app['PROPORTION_LIFE_EMPLOYED'] = app['DAYS_EMPLOYED'] / app['DAYS_BIRTH']
app['INCOME_TO_CREDIT_RATIO'] = app['AMT_INCOME_TOTAL'] / app['AMT_CREDIT'] 
app['INCOME_TO_ANNUITY_RATIO'] = app['AMT_INCOME_TOTAL'] / app['AMT_ANNUITY']
app['INCOME_TO_ANNUITY_RATIO_BY_AGE'] = app['INCOME_TO_ANNUITY_RATIO'] * app['DAYS_BIRTH']
app['CREDIT_TO_ANNUITY_RATIO'] = app['AMT_CREDIT'] / app['AMT_ANNUITY']
app['CREDIT_TO_ANNUITY_RATIO_BY_AGE'] = app['CREDIT_TO_ANNUITY_RATIO'] * app['DAYS_BIRTH']
app['INCOME_TO_FAMILYSIZE_RATIO'] = app['AMT_INCOME_TOTAL'] / app['CNT_FAM_MEMBERS']

# Add indicator columns for empty values
for col in app:
    if col!='Test' and col!='TARGET':
        app_null = app[col].isnull()
        if app_null.sum()>0:
            app[col+'_ISNULL'] = app_null

# Label encoder
le = LabelEncoder()

# Label encode binary fearures in training set
for col in app: 
    if col!='Test' and col!='TARGET' and app[col].dtype==object and app[col].nunique()==2:
        if col+'_ISNULL' in app.columns: #missing values here?
            app.loc[app[col+'_ISNULL'], col] = 'NaN'
        app[col] = le.fit_transform(app[col])
        if col+'_ISNULL' in app.columns: #re-remove missing vals
            app.loc[app[col+'_ISNULL'], col] = np.nan            

# Get categorical features to encode
cat_features = []
for col in app: 
    if col!='Test' and col!='TARGET' and app[col].dtype==object and app[col].nunique()>2:
        cat_features.append(col)

# One-hot encode categorical features in train set
app = pd.get_dummies(app, columns=cat_features)

# Hash columns
hashes = dict()
for col in app:
    hashes[col] = sha256(app[col].values).hexdigest()
    
# Get list of duplicate column lists
Ncol = app.shape[1] #number of columns
dup_list = []
dup_labels = -np.ones(Ncol)
for i1 in range(Ncol):
    if dup_labels[i1]<0: #if not already merged,
        col1 = app.columns[i1]
        t_dup = [] #list of duplicates matching col1
        for i2 in range(i1+1, Ncol):
            col2 = app.columns[i2]
            if ( dup_labels[i2]<0 #not already merged
                 and hashes[col1]==hashes[col2] #hashes match
                 and app[col1].equals(app[col2])): #cols are equal
                #then this is actually a duplicate
                t_dup.append(col2)
                dup_labels[i2] = i1
        if len(t_dup)>0: #duplicates of col1 were found!
            t_dup.append(col1)
            dup_list.append(t_dup)
        
# Merge duplicate columns
for iM in range(len(dup_list)):
    new_name = 'Merged'+str(iM)
    app[new_name] = app[dup_list[iM][0]].copy()
    app.drop(columns=dup_list[iM], inplace=True)
    #print('Merged', dup_list[iM], 'into', new_name)

# Split data back into test + train
train = app.loc[~app['Test'], :]
test = app.loc[app['Test'], :]

# Ensure all data is stored as floats
train = train.astype(np.float32)
test = test.astype(np.float32)

# Target labels
train_y = train['TARGET']

# Remove test/train indicator column and target column
train.drop(columns=['Test', 'TARGET'], inplace=True)
test.drop(columns=['Test', 'TARGET'], inplace=True)

test.reset_index(inplace=True)
train.reset_index(inplace=True)

test_orig = pd.read_csv(path + 'test_sample.csv')
train['ID']=train_ids
#test.drop(columns=['SK_ID_CURR'])
test['ID']=test_ids

# Make SK_ID_CURR the index
train.set_index('ID', inplace=True)
test.set_index('ID', inplace=True)

train.drop(columns=['index'], inplace=True)
test.drop(columns=['index'], inplace=True)

train.to_csv(path +'train_encoded.csv', index=False)
# test_encoded=test[['SK_ID_CURR','EXT_SOURCE_3','EXT_SOURCE_2','EXT_SOURCE_1','DAYS_EMPLOYED','AMT_INCOME_TOTAL','AMT_CREDIT','AMT_ANNUITY']]
# test_encoded.to_csv(path +'test_encoded_clean.csv', index=False)
test.to_csv(path +'test_encoded.csv', index=False)

/tmp/ipykernel_51528/1172781550.py:41: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  app = train.append(test, ignore_index=True)
